In [26]:
import pandas as pd
import numpy as np

I'll start by Integrating the 2018 Census Datasets together.

In [5]:
# read in all 6 2018 Census Datasets
C18_I1 = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Individual_part_1_data_by_SA1_.csv')
C18_I2 = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Individual_part_2_data_by_SA1_.csv')
C18_I3a = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Individual_part_3a_data_by_SA1_.csv')
C18_I3b = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Individual_part_3b_data_by_SA1_.csv')
C18_HH = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Household_data_by_SA1.csv')
C18_D = pd.read_csv('NZ_Data/NZ_2018_census_data/2018_Census_Dwelling_data_by_SA1.csv')

In [12]:
# merge all of the datasets on the SA12018_V1_00 column
dfs = [C18_I1, C18_I2, C18_I3a, C18_I3b, C18_HH, C18_D]

C18 = pd.concat(dfs, join='inner', axis=1)

In [22]:
# remove redundant columns
df = C18
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df = df.drop(['Shape__Area', 'Shape__Length'], axis= 1)

In [32]:
# replace negative values with NaN
df_numeric = df.select_dtypes(include=[np.number])
df_numeric = df_numeric.where(lambda x: x > 0, np.nan)

In [33]:
numeric_cols = df_numeric.columns.values
df = df.drop(columns=numeric_cols)
df = pd.concat([df, df_numeric], axis = 1)

In [37]:
# this is the complete clean 2018 Census Dataset
C18 = df

Now we need to integrate the NZDep2018 data with the 2018 Census data

In [38]:
# read in the NZDep2018 Data
NZDep18 = pd.read_csv('NZ_Data/EHINZ_data/nzdep2018-statistical-area-1-sa1-data-730395.csv')

In [46]:
# Merge this data with the 2018 Census Data and drop the redundant column
Integrated_df = pd.merge(C18, NZDep18, how='outer', left_on='SA12018_V1_00', right_on='SA12018_code')
Integrated_df = Integrated_df.drop('SA12018_code', axis = 1)

Now we need to prepare the Social Vulnerability Indicator Data to be integrated

In [72]:
# read in the dataframe
SVI18 = pd.read_csv('NZ_Data/EHINZ_data/SA2_svi.csv',low_memory=False)

I want to drop every count and denom column and remove redundent columns. I also want to prepend 'SVI18_' to each column name

In [73]:
cols = list(SVI18.columns)

In [74]:
to_drop = ['SVI18_Unnamed: 0', 'SVI18_SA2 code.1', 'SVI18_URPop', 'SVI18_households']
for i in range(0,len(cols)):
    cols[i] = 'SVI18_' + cols[i].strip()
    if "count" in cols[i] or "denom" in cols[i] or 'total' in cols[i] or 'unknown' in cols[i]:
        to_drop.append(cols[i])

In [75]:
SVI18.columns = cols
df = SVI18
df = df.drop(to_drop, axis=1)

In [88]:
df = df.drop(2145)

In [89]:
SVI18 = df

In [92]:
codes = list(SVI18['SVI18_SA2 code'])
for i in range(0, len(codes)):
    codes[i] = int(codes[i])

In [93]:
SVI18['SVI18_SA2_code'] = codes

In [95]:
df = pd.merge(Integrated_df, SVI18, left_on='SA22018_code', right_on='SVI18_SA2_code', how='outer', validate='m:1')

In [98]:
df = df.drop('SVI18_SA2 code', axis=1)

In [99]:
Integrated_df = df

Last, we need to prepare the Industry 2022 Data

In [101]:
# import the data
IND2022 = pd.read_csv('NZ_Data/economic_data/JOB12570_BD_NZ_sa1_ind6_sizegrp_2022.csv', low_memory=False)

In [103]:
df = IND2022

In [108]:
df = df.drop(df[df['ANZSIC06_Code'] == 'Total'].index).drop(df[df['Geography_Code'] == 'Total'].index).reset_index(drop=True)

In [123]:
col_list = ['SA1_code', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']
d = {}
codes = list(set(df['Geography_Code']))
length = len(codes)
for i in col_list:
    d[i] = np.full(shape=length, fill_value=np.nan)

In [124]:
for i in range(0, length):
    codes[i] = int(codes[i])
    
codes = sorted(codes)

In [125]:
d['SA1_code'] = codes

In [130]:
# iterate through the rows of the df
i = 0
code = codes[i]
for index, row in df.iterrows():
    if int(row['Geography_Code']) == code:
        division = row['ANZSIC06_Code'][0]
        if np.isnan(d[division][i]):
            d[division][i] = row['Employee_Count']
        else:
            d[division][i] = d[division][i] + row['Employee_Count']
    else:
        i = i + 1
        code = codes[i]
        division = row['ANZSIC06_Code'][0]
        if np.isnan(d[division][i]):
            d[division][i] = row['Employee_Count']
        else:
            d[division][i] = d[division][i] + row['Employee_Count']

In [134]:
df2 = pd.DataFrame.from_dict(d)

Now that we have the dataframe with the desired data (total employee counts in each industry division by SA1 district, lets rename the columns to make it more understandable

In [139]:
new_names = ['SA1_code', 'A_Agr', 'B_Mining', 'C_Manufact', 'D_Electric', 'E_Constr', 'F_Wholesale_Tr','G_Retail_Tr', 'H_Accomodat_Food', 'I_Trans_Post_Warehouse', 'J_Info_Media_Telecom',
             'K_Finance_Insure', 'L_Rent_Hire_RealEst', 'M_Proff_Scien_Tech', 'N_Admin_Support', 'O_Public_Admin_Safe', 'P_Edu_Train', 'Q_Health_Soc_Serve', 'R_Art_Rec', 'S_Other']
# Add Ind22_ to beginning of each col name
for i in range(1, len(new_names)):
    new_names[i] = 'Ind22_' + new_names[i]

In [141]:
df2.columns = new_names

In [143]:
IND2022 = df2

Now it is time to merge the final dataframe with the rest

In [160]:
df3 = pd.merge(Integrated_df, IND2022, left_on='SA12018_V1_00', right_on='SA1_code', how='outer', indicator=True)

In [165]:
ind = list(df3['_merge'])
for i in range(0, len(ind)):
    if ind[i] == 'left_only':
        ind[i] = '2018'
df3["Year"] = ind

In [167]:
df3 = df3.drop('_merge', axis=1)

In [169]:
Integrated_df = df3

In [ ]:
# drop broad birth place data becuase there is none
to_drop = ['C18_Birthpl_Brd_NZ','C18_Birthpl_Brd_Australia','C18_Birthpl_Brd_Pacific_Is','C18_Birthpl_Brd_UK_and_Ireland', 'C18_Birthpl_Brd_Europe','C18_Birthpl_Brd_North_America',
            'C18_Birthpl_Brd_Asia', 'C18_Birthpl_Brd_MidEast_Africa', 'C18_Birthpl_Brd_Other', 'C18_Birthpl_Brd_Total_stated', 'C18_Birthpl_Brd_NEI', 'C18_Birthpl_Brd_Total']
df = df.drop(to_drop, axis = 1)

In [170]:
Integrated_df.to_csv('NZ_Data/integrated_data.csv')